In [1]:
import tkinter as tk
from tkinter import messagebox
from tkinter import *
import cv2
import os
from PIL import Image
import numpy as np
import mysql.connector

In [3]:
window=tk.Tk()
window.title("Sistem Pengenalan Wajah")
window.configure(background='gray')
message = tk.Label(window, text="Griya ALKA", bg="gray", fg="black", font=('times', 30, 'italic bold '))
message.grid(column=1,row=0)
message = tk.Label(window, text="Input Data!", bg="gray", fg="black", font=('arial', 13, 'italic'))
message.grid(column=0,row=1)
l1=tk.Label(window,text="Nama",bg='gray',font=("arial",20))
l1.grid(column=0, row=2)
t1=tk.Entry(window,width=50,bd=5)
t1.grid(column=1, row=2)

l2=tk.Label(window,text="Alamat",bg='gray',font=("arial",20))
l2.grid(column=0, row=3)
t2=tk.Entry(window,width=50,bd=5)
t2.grid(column=1, row=3)

l3=tk.Label(window,text="Sekolah",bg='gray',font=("arial",20))
l3.grid(column=0, row=4)
t3=tk.Entry(window,width=50,bd=5)
t3.grid(column=1, row=4)

def capture():
    if(t1.get()=="" or t2.get()=="" or t3.get()==""):
        messagebox.showinfo('Info','Lengkapi form diatas!')
    else:
        mydb=mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="",
        database="peserta_bimbingan"
        )
        mycursor=mydb.cursor()
        mycursor.execute("SELECT * from peserta_tabel")
        myresult=mycursor.fetchall()
        id=1
        for x in myresult:
            id+=1
        sql="insert into peserta_tabel(id,Nama,Alamat,Sekolah) values(%s,%s,%s,%s)"
        val=(id,t1.get(),t2.get(),t3.get())
        mycursor.execute(sql,val)
        mydb.commit()
        face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        def face_cropped(img):
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        
        
            if faces is ():
                return None
            for (x,y,w,h) in faces:
                cropped_face = img[y:y+h,x:x+w]
            return cropped_face
    
        cap = cv2.VideoCapture(0)
        img_id = 0
    
        while True:
            ret, frame = cap.read()
            if face_cropped(frame) is not None:
                img_id+=1
                face = cv2.resize(face_cropped(frame), (200,200))
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
                cv2.imwrite(file_name_path, face)
                cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            
                cv2.imshow("Cropped face", face)
            
            if cv2.waitKey(1)==13 or int(img_id)==20:
                break
            
        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo('Result','Capturing Wajah Selesai!')

b1=tk.Button(window,text="Capture Wajah",font=("Algerian",20),bg='gray',fg='black',command=capture)
b1.grid(column=0, row=7)

def traindata():
    data_dir = "C:/Users/ASUS/Desktop/fr/data"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        
        faces.append(imageNp)
        ids.append(id)
        
    ids = np.array(ids)
    
    
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
    messagebox.showinfo('Info','Training Data Berhasil!')

b2=tk.Button(window,text="Training Data",font=("Algerian",20),bg='gray',fg='black',command=traindata)
b2.grid(column=1, row=7)

def check():
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
        color=(0,255,0)

        coords=[]

        for (x,y,w,h) in features:
            cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 )
        
            id, pred = clf.predict(gray_img[y:y+h,x:x+w])
            confidence = int(100*(1-pred/300))

            mydb=mysql.connector.connect(
            host="localhost",
            user="root",
            passwd="",
            database="peserta_bimbingan"
            )
            mycursor=mydb.cursor()
            mycursor.execute("select Nama from peserta_tabel where id="+str(id))
            s = mycursor.fetchone()
            s = ''+''.join(s)
        
            if confidence>70:
                cv2.putText(img,s,(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,2,cv2.LINE_AA) 
            else:
                cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2, cv2.LINE_AA)
            coords=[x,y,w,h]
        return coords
            
    def recognize(img,clf,faceCascade):
        coords = draw_boundary(img,faceCascade,1.1,10,(255,255,255),"Face",clf)
        return img

    
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    video_capture = cv2.VideoCapture(0)
    while True:
        ret,img = video_capture.read()
        img=  recognize(img,clf,faceCascade)
        cv2.imshow("face Detection", img)
    
        if cv2.waitKey(1)==13:
            break

    video_capture.release()
    cv2.destroyAllWindows()

b3=tk.Button(window,text="Recognize",font=("Algerian",20),bg='gray',fg='black',command=check)
b3.grid(column=2, row=7)

window.geometry('800x300')
window.mainloop()

<>:49: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:49: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\ASUS\AppData\Local\Temp/ipykernel_4932/939102032.py:49: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():
